In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('stroke_balanced_sample.csv')


target = df['stroke']
features = df.drop(columns=['stroke'])


In [3]:
features_sample = features.head(100)
target_sample = target.head(100)

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = "mistral:latest")

def predict_stroke_llm(row, llm):
    one_shot_example = """
You are a medical AI assistant. Given the following patient data, your task is to predict the likelihood of a stroke. 
Provide your response as either 0 (no stroke) or 1 (stroke), and explain your reasoning based on the data.

Example Patient:
- Gender: Female
- Age: 67
- Hypertension: 0
- Heart Disease: 1
- Ever Married: Yes
- Work Type: Private
- Residence Type: Urban
- Average Glucose Level: 228.69
- BMI: 36.6
- Smoking Status: formerly smoked

Prediction: 1 - High glucose level, heart disease, and age indicate high stroke risk.

Now evaluate the following patient:

Patient data:
- Gender: {gender}
- Age: {age}
- Hypertension: {hypertension}
- Heart Disease: {heart_disease}
- Ever Married: {ever_married}
- Work Type: {work_type}
- Residence Type: {residence_type}
- Average Glucose Level: {glucose}
- BMI: {bmi}
- Smoking Status: {smoking}

Prediction:
""".format(
        gender=row['gender'],
        age=row['age'],
        hypertension=row['hypertension'],
        heart_disease=row['heart_disease'],
        ever_married=row['ever_married'],
        work_type=row['work_type'],
        residence_type=row['Residence_type'],
        glucose=row['avg_glucose_level'],
        bmi=row['bmi'],
        smoking=row['smoking_status']
    )

    response = llm.invoke([
        {"role": "system", "content": "You are a medical expert specialized in stroke prediction."},
        {"role": "user", "content": one_shot_example}
    ])

    output = response.content.strip()

    # Naive binary extraction
    return int('1' in output.split('\n')[0])

In [5]:
preds = features_sample.apply(lambda row: predict_stroke_llm(row, llm), axis=1)

In [6]:
acc = accuracy_score(target_sample, preds)
print(f"Accuracy: {acc:.2f}")

Accuracy: 0.52
